<a href="https://colab.research.google.com/github/peremartra/llama-glu-expansion-pruning/blob/main/notebooks/02_Evaluate_1B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLU Pruning Research - Llama-3.2-1B-Instruct Evaluation
## 02 - Comprehensive Benchmark Suite Evaluation

### Exploring GLU Expansion Ratios in Llama-3.2 Models
by [Pere Martra](https://github.com/peremartra)

[![Paper](https://img.shields.io/badge/OSF-Paper-blue?logo=osf&logoColor=white)](https://doi.org/10.31219/osf.io/qgxea)
[![GitHub](https://img.shields.io/badge/⭐_Star-OptiPFair-orange?logo=github&logoColor=white)](https://github.com/peremartra/optipfair)
[![PyPI](https://img.shields.io/pypi/v/optipfair?logo=python&logoColor=white&label=v)](https://pypi.org/project/optipfair/)

**Repository:** [github.com/peremartra/llama-glu-expansion-pruning](https://github.com/peremartra/llama-glu-expansion-pruning)

---

**Colab Environment:** GPU L4 (or T4)

**Models to Evaluate:**
* Llama-3.2-1B-Instruct (base) - Baseline
* Llama-3.2-1B-I-pruned-40% (140% expansion) ⭐ Star model
* Llama-3.2-1B-I-pruned-60% (60% expansion)

**Benchmarks (5 total):**
* Lambada-OpenAI (0-shot)
* MMLU (5-shot)
* TruthfulQA MC1/MC2 (0-shot)
* GSM8K (5-shot CoT)
* IfEval (0-shot)
* leaderboard_musr (0-shot)

**Estimated Runtime:** ~4-5 hours total

---

## 📋 Notebook Objective

GLU expansion pruning (using the MAW method) appears to remove the "capacity for digression" or "computational noise" from the base model. This harms memorization (mmlu/gsm8k) but improves tasks requiring literal adherence to instructions (ifeval/musr).

---

**Note:** This evaluation uses the MAW (Maximum Absolute Weight) neuron selection method, validated in Notebook 00 as the optimal approach for GLU architectures.

---

# 1. Setup & Installation

In [1]:
# Install required libraries
!pip install -q optipfair
!pip install -q lm-eval
!pip install -q langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/llama-glu-expansion-pruning/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

✅ utils.py downloaded successfully


In [4]:
# Import core libraries and utilities
import torch
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

# Import our utility functions
from utils import (
    EXPERIMENT_CONFIG,
    BENCHMARKS_INSTRUCT,
    load_or_create_model,
    run_robust_evaluation,
    clear_gpu_cache,
    get_model_stats,
    format_results_table
)

print("✅ All imports successful")
print(f"📱 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ All imports successful
📱 Device: GPU
   GPU: NVIDIA L4
   Memory: 23.8 GB


# 2. Configuration & Planning

This section filters the experiment configuration for 1B models and displays the evaluation plan.

In [5]:
# Filter configuration for 1B models only
models_1b = [
    config for config in EXPERIMENT_CONFIG
    if "1B-I" in config["base_model"] and "3B" not in config["base_model"]
]

print(f"\n{'='*70}")
print("📊 EVALUATION PLAN: Llama-3.2-1B-Instruct Family")
print(f"{'='*70}\n")

print(f"Total models to evaluate: {len(models_1b) + 1}")  # +1 for base model
print(f"Benchmarks per model: {len(BENCHMARKS_INSTRUCT)}")
print(f"Total evaluations: {(len(models_1b) + 1) * len(BENCHMARKS_INSTRUCT)}")
print(f"Estimated runtime: ~4-5 hours\n")

# Display models table
print("Models to evaluate:")
print("-" * 70)
print(f"{'Model':<30} {'Pruning':<10} {'Star':<6}")
print("-" * 70)
print(f"{'Llama-3.2-1B-Instruct (baseline)':<30} {'0%':<10} {'300%':<12} {'N/A':<6}")
for config in models_1b:
    model_name = config['hf_repo_id'].split('/')[-1]
    pruning = f"{config['pruning_pct']}%"
    star = "⭐ Yes" if config['is_star'] else "No"
    print(f"{model_name:<30} {pruning:<10} {star:<6}")
print("-" * 70)

# Display benchmarks
print("\nBenchmarks to run:")
print("-" * 70)
for i, task in enumerate(BENCHMARKS_INSTRUCT, 1):
    task_name = task['name']
    fewshot = f"{task['num_fewshot']}-shot"
    print(f"{i:2d}. {task_name:<25} {fewshot}")
print("-" * 70)

print("\n⚙️  Configuration:")
print(f"   - Neuron selection method: MAW (Maximum Absolute Weight)")
print(f"   - Checkpointing: Enabled (per-task granularity)")
print(f"   - Model creation: On-the-fly pruning (no pre-creation needed)")
print(f"   - Error handling: Skip failed tasks and continue\n")


📊 EVALUATION PLAN: Llama-3.2-1B-Instruct Family

Total models to evaluate: 4
Benchmarks per model: 7
Total evaluations: 28
Estimated runtime: ~4-5 hours

Models to evaluate:
----------------------------------------------------------------------
Model                          Pruning    Star  
----------------------------------------------------------------------
Llama-3.2-1B-Instruct (baseline) 0%         300%         N/A   
Llama-3.2-1B-I-pruned-10pct    10%        No    
Llama-3.2-1B-I-pruned-40pct    40%        ⭐ Yes 
Llama-3.2-1B-I-pruned-60pct    60%        No    
----------------------------------------------------------------------

Benchmarks to run:
----------------------------------------------------------------------
 1. leaderboard_musr          0-shot
 2. truthfulqa_mc1            0-shot
 3. truthfulqa_mc2            0-shot
 4. lambada_openai            0-shot
 5. mmlu                      5-shot
 6. gsm8k                     5-shot
 7. ifeval                    0-shot
--

In [6]:
# Setup checkpoint paths
CHECKPOINT_DIR = "/content/drive/MyDrive/glu_pruning/checkpoints/1bI"
RESULTS_DIR = "/content/drive/MyDrive/glu_pruning/results"

# Create directories if they don't exist
Path(CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

print(f"✅ Checkpoint directory: {CHECKPOINT_DIR}")
print(f"✅ Results directory: {RESULTS_DIR}")

# Define checkpoint paths for each model
checkpoint_paths = {
    "baseline": f"{CHECKPOINT_DIR}/llama_3.2_1b_I_baseline.json",
    "40pct": f"{CHECKPOINT_DIR}/llama_3.2_1b_I_pruned_40pct.json",
    "60pct": f"{CHECKPOINT_DIR}/llama_3.2_1b_I_pruned_60pct.json",
}

print("\nCheckpoint files:")
for key, path in checkpoint_paths.items():
    exists = "✅ Exists" if Path(path).exists() else "🆕 New"
    print(f"   {key:<10}: {exists}")

✅ Checkpoint directory: /content/drive/MyDrive/glu_pruning/checkpoints/1bI
✅ Results directory: /content/drive/MyDrive/glu_pruning/results

Checkpoint files:
   baseline  : ✅ Exists
   40pct     : ✅ Exists
   60pct     : ✅ Exists


# 3. Baseline Evaluation

Evaluate the original Llama-3.2-1B model to establish performance baseline.

In [ ]:
print(f"\n{'='*70}")
print("📊 PHASE 1: BASELINE EVALUATION")
print(f"{'='*70}\n")

BASE_MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

# Load base model
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"Loading base model: {BASE_MODEL_ID}...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    #dtype=torch.float16, #T4
    dtype=torch.bfloat16, #A100 <- Used for experiments
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Model loaded successfully")

# Display model statistics
base_stats = get_model_stats(base_model)
print(f"\n📈 Model Statistics:")
print(f"   Parameters: {base_stats['total_parameters']:,}")
print(f"   Size: {base_stats['size_gb']:.2f} GB")

# Run evaluation with checkpointing
baseline_results = run_robust_evaluation(
    model=base_model,
    tokenizer=tokenizer,
    tasks=BENCHMARKS_INSTRUCT,
    checkpoint_path=checkpoint_paths["baseline"],
    model_name="Llama-3.2-1B-I-baseline"
)

print(f"\n{'='*70}")
print("✅ BASELINE EVALUATION COMPLETED")
print(f"{'='*70}\n")

# Display results summary
print("Results Preview:")
print(format_results_table(baseline_results))

# Clear memory
del base_model
clear_gpu_cache()

print("\n🧹 Memory cleared, ready for pruned models")


📊 PHASE 1: BASELINE EVALUATION

Loading base model: meta-llama/Llama-3.2-1B-Instruct...


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

# 4. Pruned Models Evaluation Loop

Evaluate the three pruned variants (20%, 40%, 60%) using on-the-fly pruning with OptiPFair.

In [ ]:
print(f"\n{'='*70}")
print("📊 PHASE 2: PRUNED MODELS EVALUATION")
print(f"{'='*70}\n")

# Store all results for final comparison
all_results = {
    "baseline": baseline_results
}

# Evaluate each pruned model
for i, config in enumerate(models_1b, 1):
    model_name = config['hf_repo_id'].split('/')[-1]
    pruning_pct = config['pruning_pct']
    is_star = config['is_star']

    print(f"\n{'─'*70}")
    print(f"🔄 EVALUATING MODEL {i}/{len(models_1b)}: {model_name}{pruning_pct} ")
    print(f"   Pruning: {pruning_pct}% |  Star: {'⭐' if is_star else 'No'}")
    print(f"{'─'*70}\n")

    try:
        # Load or create model using utility function
        model, tokenizer, stats = load_or_create_model(config, device="auto")

        # Display model statistics
        print(f"\n📈 Model Statistics:")
        print(f"   Parameters: {stats['total_parameters']:,}")
        print(f"   Size: {stats['size_gb']:.2f} GB")
        if 'pruning_stats' in stats:
            print(f"   Reduction: {stats['pruning_stats']['percentage_reduction']:.2f}%")
        print(f"   Source: {stats['source']}\n")

        # Determine checkpoint key
        checkpoint_key = f"{pruning_pct}pct"

        # Run evaluation with checkpointing
        results = run_robust_evaluation(
            model=model,
            tokenizer=tokenizer,
            tasks=BENCHMARKS_INSTRUCT,
            checkpoint_path=checkpoint_paths[checkpoint_key],
            model_name=model_name
        )

        # Store results
        all_results[checkpoint_key] = results

        print(f"\n✅ {model_name}{pruning_pct} evaluation completed")
        print("\nResults Preview:")
        print(format_results_table(results))

        # Clear memory before next model
        del model
        clear_gpu_cache()

    except Exception as e:
        print(f"\n❌ ERROR evaluating {model_name}: {str(e)}")
        print("   Continuing with next model...\n")
        clear_gpu_cache()
        continue

print(f"\n{'='*70}")
print("✅ ALL PRUNED MODELS EVALUATED")
print(f"{'='*70}\n")

# 5. Results Consolidation & Export

Consolidate all evaluation results and export to CSV for analysis.

In [ ]:
# Import necessary libraries for this cell
import os
import json
import glob
import re
import pandas as pd
from datetime import datetime

print(f"\n{'='*70}")
print("📊 CONSOLIDATING RESULTS (DYNAMIC FILE-BASED)")
print(f"{'='*70}\n")

# --- Directory Setup ---
# Ensure CHECKPOINT_DIR is defined (it should be defined in a previous cell)
# This is where the individual JSON results are.
# Example: CHECKPOINT_DIR = "/content/drive/MyDrive/glu_pruning/checkpoints/1b"
if 'CHECKPOINT_DIR' not in globals():
    print("⚠️ Warning: CHECKPOINT_DIR not set. Using default './checkpoints/1bI'")
    CHECKPOINT_DIR = "./checkpoints/1bI"

# Ensure RESULTS_DIR is defined (it should be defined in a previous cell)
# This is where the consolidated CSV will be saved.
# Example: RESULTS_DIR = "/content/drive/MyDrive/glu_pruning/results"
if 'RESULTS_DIR' not in globals():
    print("⚠️ Warning: RESULTS_DIR not set. Using default './results'")
    RESULTS_DIR = "./results"
# --- End Directory Setup ---


# Prepare data for DataFrame
consolidated_data = []

# --- Dynamic Loading ---
# 1. Find all individual 1B model result files
# *** THIS IS THE CORRECTED LINE: Using CHECKPOINT_DIR ***
json_files = glob.glob(f"{CHECKPOINT_DIR}/llama_3.2_1b_I_*.json")

# 2. Exclude any aggregate/summary files
json_files = [
    f for f in json_files
    if "results" not in os.path.basename(f) and "complete" not in os.path.basename(f)
]

print(f"Searching for results in: {CHECKPOINT_DIR}")
print(f"Found {len(json_files)} individual result files to process:")

# 3. Process each model's result file
for json_path in sorted(json_files):
    print(f"  -> Processing: {os.path.basename(json_path)}")
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"    ⚠️ Warning: Could not read or parse file. Error: {e}")
        continue

    # Extract metadata and results
    metadata = data.get("metadata", {})
    model_name_from_file = metadata.get("model_name", "Unknown Model")

    results = data.get("results", {})
    if not results:
        print(f"    ⚠️ Warning: No 'results' found in file. Skipping.")
        continue

    # --- Dynamically derive info from metadata ---
    pruning_pct = 0
    is_star = False

    # Parse model name to get pruning percentage and display name
    if "baseline" in model_name_from_file:
        display_name = "Llama-3.2-1B-Instruct"
        pruning_pct = 0
    else:
        # Use regex to find pruning percentage
        match = re.search(r'pruned-(\d+)pct', model_name_from_file)
        if match:
            pruning_pct = int(match.group(1))
            display_name = f"Llama-3.2-1B-I-pruned-{pruning_pct}%"
        else:
            display_name = model_name_from_file # Fallback

    # Star logic (as per original project spec/hardcoded cell)
    # 40% is the "star" model for the 1B variant
    if pruning_pct == 40:
        is_star = True
    # --- End dynamic info derivation ---

    # Process each task for this model
    for task_name, metrics in results.items():
        row = {
            "model": display_name,
            "pruning_pct": pruning_pct,
            "is_star": is_star,
            "task": task_name,
        }

        # Add all metrics from this task
        for metric_name, value in metrics.items():
            # Convert string values to float where possible
            try:
                row[metric_name] = float(value)
            except (ValueError, TypeError):
                row[metric_name] = value

        consolidated_data.append(row)

# --- End Dynamic Loading ---

In [ ]:
# Create DataFrame
df = pd.DataFrame(consolidated_data)

# Sort by pruning_pct and then task to ensure consistent order
if not df.empty:
    df = df.sort_values(by=["pruning_pct", "task"]).reset_index(drop=True)

print(f"\n✅ Consolidated {len(df)} result rows")
print(f"   Models: {df['model'].nunique()}")
print(f"   Tasks: {df['task'].nunique()}")
if 'model' in df.columns:
    print(f"   Metrics per task: {len(df.columns) - 4}")  # Exclude metadata columns
else:
    print("   No data consolidated.")

# Display summary
print("\nDataFrame Preview:")
print(df.head(10))

# Save to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = f"{RESULTS_DIR}/llama_1b_I_results_{timestamp}.csv"
df.to_csv(csv_path, index=False)

print(f"\n💾 Results saved to: {csv_path}")

# Also save a "latest" version for easy access
latest_path = f"{RESULTS_DIR}/llama_1b_I_results_latest.csv"
df.to_csv(latest_path, index=False)
print(f"💾 Latest results: {latest_path}")

print(f"\n{'='*70}")
print("✅ EVALUATION COMPLETE - ALL RESULTS SAVED")
print(f"{'='*70}\n")

# 6. Quick Analysis & Visualization

Generate quick insights to decide which models merit uploading to HuggingFace Hub.

In [ ]:
print(f"\n{'='*70}")
print("📈 QUICK ANALYSIS: Performance vs. Pruning Level")
print(f"{'='*70}\n")

# This cell assumes the 'df' DataFrame was created in the previous cell.

# Calculate average performance degradation per model
# Focus on key metrics: accuracy for classification, perplexity for generation

summary_metrics = []

# --- Dynamic Analysis ---
# Group by the model-level info we created in the previous cell
# This replaces the hardcoded 'model_info' dictionary
try:
    grouped = df.groupby(['model', 'pruning_pct', 'is_star'])
except KeyError:
    print("❌ Error: 'df' DataFrame not found or is missing required columns.")
    print("   Please ensure the previous consolidation cell was run successfully.")
    # Create an empty df to avoid crashing the rest of the cell
    grouped = pd.DataFrame().groupby(['model', 'pruning_pct', 'is_star'])

print(f"Analyzing {len(grouped)} unique models found in the DataFrame...")

for (model_name, pruning, is_star_bool), model_df in grouped:
    # 'model_df' is now the DataFrame for this specific model

    # Extract key metrics
    # We look for 'accuracy' (from boolq, mmlu, etc.)
    accuracies = model_df['accuracy'].dropna()
    # We look for 'perplexity' (from lambada_openai)
    # Note: wikitext 'word_perplexity,none' is not used here for simplicity
    perplexities = model_df['perplexity'].dropna()

    summary = {
        "model": model_name,
        "pruning": pruning,
        "star": "⭐" if is_star_bool else "",
        "avg_accuracy": accuracies.mean() if len(accuracies) > 0 else None,
        "avg_perplexity": perplexities.mean() if len(perplexities) > 0 else None,
        "num_tasks": len(model_df),
    }

    summary_metrics.append(summary)

# --- End Dynamic Analysis ---

if not summary_metrics:
    print("\nNo summary metrics to display. Skipping analysis.")
else:
    summary_df = pd.DataFrame(summary_metrics)
    # Sort by pruning percentage to ensure a logical order
    summary_df = summary_df.sort_values(by="pruning").reset_index(drop=True)

    print("\nPerformance Summary:")
    print("-" * 90)
    print(summary_df.to_string(index=False, float_format="%.4f"))
    print("-" * 90)

    # Calculate degradation vs baseline
    # Find baseline row (pruning == 0)
    baseline_row = summary_df.loc[summary_df['pruning'] == 0]

    if baseline_row.empty:
        print("\n⚠️ Baseline model (pruning=0) not found. Cannot calculate degradation.")
    else:
        baseline_acc = baseline_row['avg_accuracy'].values[0]
        baseline_ppl = baseline_row['avg_perplexity'].values[0]

        print(f"\nDegradation vs. Baseline (Acc: {baseline_acc:.4f}, PPL: {baseline_ppl:.2f}):")
        print("-" * 90)

        for _, row in summary_df.iterrows():
            if row['pruning'] == 0:
                continue

            acc_delta_str = "N/A"
            if row['avg_accuracy'] is not None and baseline_acc is not None and baseline_acc != 0:
                acc_delta = ((row['avg_accuracy'] - baseline_acc) / baseline_acc * 100)
                acc_delta_str = f"{acc_delta:+.2f}%"

            ppl_delta_str = "N/A"
            if row['avg_perplexity'] is not None and baseline_ppl is not None and baseline_ppl != 0:
                ppl_delta = ((row['avg_perplexity'] - baseline_ppl) / baseline_ppl * 100)
                ppl_delta_str = f"{ppl_delta:+.2f}%"

            print(f"{row['model']:<35} {row['star']:>2}")
            print(f"   Accuracy:   {acc_delta_str}")
            print(f"   Perplexity: {ppl_delta_str}")
            print()

        print("-" * 90)

In [ ]:
# Visualization: Performance across pruning levels
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(summary_df['pruning'], summary_df['avg_accuracy'], marker='o', linewidth=2, markersize=8)
axes[0].axhline(y=baseline_acc, color='r', linestyle='--', alpha=0.5, label='Baseline')
axes[0].set_xlabel('Pruning Level (%)', fontsize=12)
axes[0].set_ylabel('Average Accuracy', fontsize=12)
axes[0].set_title('Accuracy vs. Pruning Level', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Highlight star model
star_idx = summary_df[summary_df['star'] == '⭐'].index[0]
axes[0].plot(summary_df.loc[star_idx, 'pruning'], summary_df.loc[star_idx, 'avg_accuracy'],
             marker='*', markersize=20, color='gold', markeredgecolor='black', markeredgewidth=1.5)

# Perplexity plot
axes[1].plot(summary_df['pruning'], summary_df['avg_perplexity'], marker='o', linewidth=2, markersize=8, color='orange')
axes[1].axhline(y=baseline_ppl, color='r', linestyle='--', alpha=0.5, label='Baseline')
axes[1].set_xlabel('Pruning Level (%)', fontsize=12)
axes[1].set_ylabel('Average Perplexity', fontsize=12)
axes[1].set_title('Perplexity vs. Pruning Level', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Highlight star model
axes[1].plot(summary_df.loc[star_idx, 'pruning'], summary_df.loc[star_idx, 'avg_perplexity'],
             marker='*', markersize=20, color='gold', markeredgecolor='black', markeredgewidth=1.5)

plt.tight_layout()
plt.savefig(f"{RESULTS_DIR}/llama_1b_I_performance_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Visualization saved to: {RESULTS_DIR}/llama_1b_I_performance_analysis.png")

# 7. Decision Matrix: Which Models to Upload?

Based on the evaluation results, determine which models should be uploaded to HuggingFace Hub for Phase 2.

In [ ]:
import numpy as np # Need numpy for nan checks
import pandas as pd # Ensure pandas is imported

print(f"\n{'='*70}")
print("🎯 DECISION MATRIX: Models for HuggingFace Hub Upload")
print(f"{'='*70}\n")

print("Evaluation Criteria:")
print("  1. Performance degradation (avg_accuracy) < 15% vs baseline")
print("  2. Outperforms or matches baseline in at least 3 tasks (Primary Metric)")
print("  3. Accuracy degradation (avg_accuracy) < 50% vs baseline")
print("  4. Sufficient parameter reduction to justify storage\n")

# --- Setup: Load BENCHMARKS_BASE and Baseline Scores (as before) ---
print("Building primary metric map from BENCHMARKS_INSTRUCT...")
TASK_PRIMARY_METRICS = {}
if 'BENCHMARKS_INSTRUCT' not in globals():
    print("="*70)
    print("❌ Error: BENCHMARKS_INSTRUCT variable not found in global scope.")
    print("   Please ensure you have run the setup cell that imports from utils.py:")
    print("   >>> from utils import BENCHMARKS_INSTRUCT")
    print("="*70)
    raise NameError("BENCHMARKS_INSTRUCT is not defined.")
else:
    print(f"✅ Found BENCHMARKS_INSTRUCT with {len(BENCHMARKS_INSTRUCT)} tasks.")

for task_spec in BENCHMARKS_INSTRUCT:
    task_name = task_spec["name"]
    if task_name == 'wikitext':
        TASK_PRIMARY_METRICS[task_name] = ('word_perplexity,none', False) # Lower is better
    elif task_name == 'lambada_openai':
        TASK_PRIMARY_METRICS[task_name] = ('perplexity', False) # Lower is better
    elif task_name == 'gsm8k':
        TASK_PRIMARY_METRICS[task_name] = ('exact_match,strict-match', True) # Higher is better
    else:
        TASK_PRIMARY_METRICS[task_name] = ('accuracy', True) # Higher is better
print(f"Built primary metric map for {len(TASK_PRIMARY_METRICS)} tasks.")

try:
    df_baseline = df[df['pruning_pct'] == 0].set_index('task')
    baseline_scores = {}
    for task, (metric, _) in TASK_PRIMARY_METRICS.items():
        if task in df_baseline.index and metric in df_baseline.columns:
            score = df_baseline.loc[task, metric]
            if not pd.isna(score):
                baseline_scores[task] = score
    print(f"Captured {len(baseline_scores)} valid scores from baseline model.")
except Exception as e:
    print(f"❌ Error: Could not get baseline scores from 'df'. {e}")
    baseline_scores = {}
# --- End Setup ---


# Decision logic
decisions = []
if 'summary_df' not in globals():
     print("❌ Error: 'summary_df' not found. Please run the previous analysis cell.")
else:
    for _, row in summary_df.iterrows():
        if row['pruning'] == 0:
            continue  # Skip baseline

        decision = {
            "model": row['model'],
            "pruning": row['pruning'],
            "star": row['star'],
        }

        acc_degradation = abs((row['avg_accuracy'] - baseline_acc) / baseline_acc * 100) if baseline_acc and not pd.isna(row['avg_accuracy']) else 999
        ppl_degradation = abs((row['avg_perplexity'] - baseline_ppl) / baseline_ppl * 100) if baseline_ppl and not pd.isna(row['avg_perplexity']) else 999

        # --- MODIFIED: Check Criterion 2 AND store task names ---
        outperform_count = 0
        outperforming_tasks = []  # <-- NEW: List to store names
        model_tasks_df = df[df['model'] == row['model']].set_index('task')

        for task, (metric, higher_is_better) in TASK_PRIMARY_METRICS.items():
            if task not in baseline_scores or task not in model_tasks_df.index:
                continue

            model_score = model_tasks_df.loc[task, metric]
            if pd.isna(model_score):
                continue

            baseline_score = baseline_scores[task]

            if higher_is_better:
                if model_score >= baseline_score:
                    outperform_count += 1
                    outperforming_tasks.append(task) # <-- NEW: Store task name
            else: # Lower is better
                if model_score <= baseline_score:
                    outperform_count += 1
                    outperforming_tasks.append(task) # <-- NEW: Store task name

        decision['outperform_count'] = outperform_count
        decision['outperforming_tasks'] = outperforming_tasks # <-- NEW: Store list
        tasks_str = ", ".join(outperforming_tasks) if outperforming_tasks else "None"
        # --- End Criterion 2 Check ---


        # --- MODIFIED: Updated Decision Logic with task list ---
        is_star = row['star'] == '⭐'
        crit_1_low_degrad = (acc_degradation < 15)
        crit_2_tasks = (outperform_count >= 3)
        crit_3_not_catastrophic = (acc_degradation < 50)

        if is_star:
            decision['upload'] = True
            decision['reason'] = f"Star model. Won/Tied: {tasks_str}"
        elif crit_1_low_degrad and crit_2_tasks:
            decision['upload'] = True
            decision['reason'] = f"Low degradation (acc: {acc_degradation:.1f}%) AND Won/Tied: {tasks_str}"
        elif crit_3_not_catastrophic and crit_2_tasks:
            decision['upload'] = True
            decision['reason'] = f"Acceptable degradation (acc: {acc_degradation:.1f}%) AND Won/Tied: {tasks_str}"
        else:
            decision['upload'] = False
            reason_parts = []
            if not crit_3_not_catastrophic:
                reason_parts.append(f"High acc degradation ({acc_degradation:.1f}%)")
            if not crit_2_tasks:
                reason_parts.append(f"Only {outperform_count} tasks won ({tasks_str})")

            if not reason_parts:
                 reason_parts.append(f"Degradation (acc: {acc_degradation:.1f}%) or task count ({outperform_count}) too low")

            decision['reason'] = " AND ".join(reason_parts)

        decisions.append(decision)

# Display decision table
print("\nUpload Decisions (Updated Logic with Task Details):")
print("-" * 140) # <-- Widen table
# <-- MODIFIED: Widen Reason column
print(f"{'Model':<35} {'Pruning':<10} {'Star':<6} {'Tasks Won/Tied':<16} {'Upload?':<10} {'Reason'}")
print("-" * 140) # <-- Widen table

for dec in decisions:
    upload_status = "✅ YES" if dec['upload'] else "❌ NO"
    print(f"{dec['model']:<35} {dec['pruning']:<10}% {dec['star']:<6} {dec.get('outperform_count', 'N/A'):<16} {upload_status:<10} {dec['reason']}")

print("-" * 140) # <-- Widen table

# Summary
models_to_upload = sum(1 for d in decisions if d['upload'])
print(f"\n📦 Total models to upload to HF Hub: {models_to_upload}/{len(decisions)}")
print(f"\n✅ PHASE 3 COMPLETE - Ready for Phase 2 (Model Factory)")

In [ ]:
print(f"\n{'='*70}")
print("💾 SAVING COMPLETE RESULTS FOR RESEARCH SHARING")
print(f"{'='*70}\n")

# --- Dynamic Model Info Setup ---
# Build a lookup for expansion rates from EXPERIMENT_CONFIG (if it exists)
exp_rate_map = {}
if 'EXPERIMENT_CONFIG' in globals():
    try:
        # --- ROBUST PARSING ---
        # Only add if all keys are present
        for cfg in EXPERIMENT_CONFIG:
            if (cfg.get("base_model", "") == "meta-llama/Llama-3.2-1B-Instruct"
                and 'pruning_pct' in cfg
                and 'expansion_rate' in cfg):
                 exp_rate_map[cfg['pruning_pct']] = cfg['expansion_rate']
    except Exception as e:
        print(f"⚠️ Warning: Could not parse EXPERIMENT_CONFIG. {e}")
exp_rate_map[0] = 300 # Baseline expansion rate

print(f"Loaded {len(exp_rate_map)} expansion rates from config (Baseline included).")

# Build 'models_evaluated' section dynamically from the 'df'
models_evaluated = {}
if 'df' not in globals():
    print("❌ Error: 'df' DataFrame not found. Cannot build results.")
    raise NameError("'df' is not defined. Please run consolidation cell.")

# Group by model to rebuild the nested result structure
grouped_by_model = df.groupby(['model', 'pruning_pct', 'is_star'])

for (model_name, pruning_pct, is_star_bool), model_df in grouped_by_model:
    model_key = f"pruned_{int(pruning_pct)}pct" if pruning_pct > 0 else "baseline"

    # Rebuild the nested results dict
    model_results_dict = {}
    for _, row in model_df.iterrows():
        task_name = row['task']
        metrics = row.drop(['model', 'pruning_pct', 'is_star', 'task'])

        task_metrics = {}
        for col, val in metrics.dropna().items():
            # Convert numpy types to native Python types
            if isinstance(val, (np.integer, np.int64)):
                task_metrics[col] = int(val)
            elif isinstance(val, (np.floating, np.float64)):
                task_metrics[col] = float(val)
            else:
                task_metrics[col] = val
        model_results_dict[task_name] = task_metrics

    # Assemble the entry for this model
    models_evaluated[model_key] = {
        "name": model_name,
        "pruning_pct": int(pruning_pct), # Cast to native int
        "expansion_rate": exp_rate_map.get(int(pruning_pct), None), # Get from map
        "is_star": bool(is_star_bool), # <--- *** CRITICAL FIX: Cast to native bool ***
        "hf_repo": (
            "meta-llama/Llama-3.2-1B-Instruct" if pruning_pct == 0
            else f"peremartra/Llama-3.2-1B-I-pruned-{int(pruning_pct)}pct"
        ),
        "results": model_results_dict
    }
print(f"Dynamically built {len(models_evaluated)} model entries from 'df'.")
# --- End Dynamic Model Setup ---


# Consolidate all data into a comprehensive JSON
complete_results = {
    "experiment_metadata": {
        "timestamp": datetime.now().isoformat(),
        "notebook": "03_Evaluate_1B_Instruct.ipynb",
        "model_family": "Llama-3.2-1B-Instruct",
        "pruning_method": "MAW (Maximum Absolute Weight)",
        #"library_versions": library_versions,
        "hardware": {
            "device": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "gpu_memory_gb": torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else None
        }
    },

    "benchmarks": [
        {"name": task["name"], "num_fewshot": task["num_fewshot"]}
        for task in BENCHMARKS_INSTRUCT
    ],

    "models_evaluated": models_evaluated, # Use dynamic model data

    "summary_statistics": {
        "baseline": {
            "avg_accuracy": float(summary_df.loc[summary_df['pruning'] == 0, 'avg_accuracy'].values[0]),
            "avg_perplexity": float(summary_df.loc[summary_df['pruning'] == 0, 'avg_perplexity'].values[0]),
        },
        "pruned_models": [
            {
                "pruning_pct": int(row['pruning']),
                "is_star": row['star'] == '⭐', # This is native bool, so it's fine
                "avg_accuracy": float(row['avg_accuracy']) if pd.notna(row['avg_accuracy']) else None,
                "avg_perplexity": float(row['avg_perplexity']) if pd.notna(row['avg_perplexity']) else None,
                "accuracy_degradation_pct": float(((row['avg_accuracy'] - baseline_acc) / baseline_acc * 100)) if baseline_acc and pd.notna(row['avg_accuracy']) else None,
                "perplexity_degradation_pct": float(((row['avg_perplexity'] - baseline_ppl) / baseline_ppl * 100)) if baseline_ppl and pd.notna(row['avg_perplexity']) else None
            }
            for _, row in summary_df.iterrows() if row['pruning'] > 0
        ]
    },

    "upload_decisions": decisions, # Use dynamic decisions

    "citation": {
        "paper": "Exploring GLU Expansion Ratios: Structured Pruning in Llama-3.2 Models",
        "author": "Pere Martra",
        "doi": "https://doi.org/10.31219/osf.io/qgxea",
        "github": "https://github.com/peremartra/llama-glu-expansion-pruning",
        "note": "Results are freely available for research purposes. Please cite the paper if you use this data."
    }
}

# --- Save to JSON ---
try:
    # Ensure RESULTS_DIR is defined
    if 'RESULTS_DIR' not in globals():
        print("❌ Error: RESULTS_DIR not defined. Defaulting to './results'")
        RESULTS_DIR = "./results"

    # Create RESULTS_DIR if it doesn't exist
    os.makedirs(RESULTS_DIR, exist_ok=True)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    json_path = f"{RESULTS_DIR}/llama_1b_I_complete_results_{timestamp}.json"
    with open(json_path, 'w') as f:
        json.dump(complete_results, f, indent=2, ensure_ascii=False)

    print(f"✅ Complete results saved to:")
    print(f"   {json_path}")

    # Also save a "latest" version
    latest_json = f"{RESULTS_DIR}/llama_1b_I_complete_results_latest.json"
    with open(latest_json, 'w') as f:
        json.dump(complete_results, f, indent=2, ensure_ascii=False)

    print(f"✅ Latest version:")
    print(f"   {latest_json}")

    # Display file size
    file_size_kb = Path(json_path).stat().st_size / 1024
    print(f"\n📊 File size: {file_size_kb:.1f} KB")

except Exception as e:
    print(f"❌ Error saving JSON files: {e}")
    print(f"   Please ensure RESULTS_DIR is defined and writeable: {RESULTS_DIR}")


print(f"\n📦 Models included: {len(complete_results['models_evaluated'])}")
print(f"📋 Benchmarks per model: {len(BENCHMARKS_INSTRUCT)}")
print(f"🔬 Total result entries: {len(df)}")

print(f"\n{'='*70}")
print("✅ COMPLETE RESULTS SAVED - Ready for research sharing")
# --- THIS IS THE CORRECTED LINE ---
print(f"{'='*70}\n")

---

## 🎓 Key Takeaways

This notebook evaluated the Llama-3.2-1B model family across 10 comprehensive benchmarks to determine:

1. **Optimal pruning level** for GLU-MLP layers
2. **Performance-efficiency trade-offs** at different expansion ratios
3. **Which models justify upload** to HuggingFace Hub


---

**Powered by OptiPFair** - Structured Pruning for GLU Architectures

If this research helps your work:
- ⭐ Star [the repo](https://github.com/peremartra/optipfair)
- 📖 Read the [documentation](https://peremartra.github.io/optipfair/)
- 🐛 Report issues or suggest features

---